In [6]:
#Imports
import sqlite3 as sql
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

In [7]:
import os
print(os.getcwd())

/Users/blakelaw/PycharmProjects/RefereeAnalysis


In [8]:
conn = sql.connect('nba.sqlite') # create connection object to database
conn

In [9]:
df = pd.read_sql('select * from combined_all2', conn)

In [10]:
df.to_csv('df.csv')

In [11]:
home_games = df[['team_abbreviation_home','game_id', 'game_date', 'official_id', 'team_id_home', 'pts_home', 'first_name', 'last_name']].rename(columns={'team_abbreviation_home': 'team_abbreviation','team_id_home': 'team_id', 'pts_home': 'pts'})
away_games = df[['team_abbreviation_away','game_id', 'game_date', 'official_id', 'team_id_away', 'pts_away','first_name', 'last_name']].rename(columns={'team_abbreviation_away': 'team_abbreviation', 'team_id_away': 'team_id', 'pts_away': 'pts'})


In [12]:
combined = pd.concat([home_games, away_games], axis=0)

In [13]:
grouped_df = combined.groupby(['official_id', 'team_abbreviation']).agg(avg_pts=('pts', 'mean'), num_games=('pts', 'size')).reset_index()

In [14]:
grouped_df

,official_id,team_abbreviation,avg_pts,num_games
0,101283,ATL,105.885246,61
1,101283,BKN,104.575000,40
2,101283,BOS,104.682540,63
3,101283,CHA,97.381818,55
4,101283,CHI,100.627119,59
...,...,...,...,...
2847,2882,SAS,105.885246,61
2848,2882,SEA,91.500000,10
2849,2882,TOR,104.353846,65
2850,2882,UTA,101.875000,56


In [15]:
expanded_df = grouped_df.merge(combined[['official_id', 'first_name', 'last_name']], on='official_id', how='left')


In [16]:
expanded_df

,official_id,team_abbreviation,avg_pts,num_games,first_name,last_name
0,101283,ATL,105.885246,61,Brian,Forte
1,101283,ATL,105.885246,61,Brian,Forte
2,101283,ATL,105.885246,61,Brian,Forte
3,101283,ATL,105.885246,61,Brian,Forte
4,101283,ATL,105.885246,61,Brian,Forte
...,...,...,...,...,...,...
4173351,2882,WAS,105.348485,66,Sean,Wright
4173352,2882,WAS,105.348485,66,Sean,Wright
4173353,2882,WAS,105.348485,66,Sean,Wright
4173354,2882,WAS,105.348485,66,Sean,Wright


In [17]:
expanded_df = expanded_df.drop_duplicates()
expanded_df = expanded_df.rename(columns={'avg_pts': 'avg_pts_ref'})

In [18]:
avg = combined.groupby('team_abbreviation')['pts'].mean().reset_index()
avg = avg.rename(columns={'pts': 'avg_score'})

In [19]:
expanded_df = expanded_df.merge(avg, on='team_abbreviation', how='left')

# Rename the 'avg_score' column from 'new_table' to 'avg_team_score'
expanded_df = expanded_df.rename(columns={'avg_score': 'avg_pts_team'})

In [20]:
expanded_df['score_difference'] = expanded_df['avg_pts_ref'] - expanded_df['avg_pts_team']
expanded_df['abs_score_difference'] = expanded_df['score_difference'].abs()

In [21]:
expanded_df_30 = expanded_df[expanded_df['num_games'] > 30].reset_index()
expanded_df_30 = expanded_df_30.sort_values(by='abs_score_difference', ascending=False)

In [22]:
diagram = expanded_df_30.head(10)[['team_abbreviation', 'first_name', 'last_name', 'num_games', 'avg_pts_team', 'avg_pts_ref', 'score_difference', 'abs_score_difference']]
diagram.insert(3, 'Full Name', diagram['first_name'] + ' ' + diagram['last_name'])

diagram = diagram.rename(columns={
    'team_abbreviation': 'Team',
    'num_games': 'Games',
    'avg_pts_team': 'Team Average',
    'avg_pts_ref': 'Ref Average',
    'score_difference': 'Difference',
    'abs_score_difference': 'Absolute Difference'
})
diagram

,Team,first_name,last_name,Full Name,Games,Team Average,Ref Average,Difference,Absolute Difference
1700,WAS,Gediminas,Petraitis,Gediminas Petraitis,33,101.944863,117.696970,15.752107,15.752107
954,MIN,Aaron,Smith,Aaron Smith,31,101.570768,115.000000,13.429232,13.429232
1677,LAC,Dedric,Taylor,Dedric Taylor,36,102.615403,115.027778,12.412375,12.412375
1699,POR,Gediminas,Petraitis,Gediminas Petraitis,36,101.471829,113.694444,12.222615,12.222615
1587,POR,Kevin,Scott,Kevin Scott,39,101.471829,112.974359,11.502530,11.502530
1704,MIL,Tyler,Ford,Tyler Ford,32,102.275704,112.937500,10.661796,10.661796
334,LAC,Kevin,Fehr,Kevin Fehr,42,102.615403,92.000000,-10.615403,10.615403
1702,IND,Tyler,Ford,Tyler Ford,34,101.374695,111.882353,10.507658,10.507658
1674,SAC,Mitchell,Ervin,Mitchell Ervin,31,103.970955,114.193548,10.222593,10.222593
1673,PHI,Mitchell,Ervin,Mitchell Ervin,32,100.098025,110.093750,9.995725,9.995725


In [23]:
Ged_Games = combined[(combined['first_name']=='Gediminas') & (combined['team_abbreviation']=='WAS')][['game_date','pts']]
WAS = combined[combined['team_abbreviation']=='WAS'][['game_date','pts']]

Ged_Games = Ged_Games.drop_duplicates()
WAS = WAS.drop_duplicates()

WAS['game_date'] = pd.to_datetime(WAS['game_date'])
Ged_Games['game_date'] = pd.to_datetime(Ged_Games['game_date'])

In [24]:
Ged_Games.to_csv('Ged_Games.csv')
WAS.to_csv('WAS.csv')

In [25]:
diff = expanded_df_30['score_difference'].round().astype(int)

In [26]:
diff.to_csv('diff.csv')

In [27]:
combined['game_date'] = pd.to_datetime(combined['game_date'])
combined['year'] = combined['game_date'].dt.year

In [28]:
avg2 = combined.groupby(['year', 'team_abbreviation'])['pts'].mean().reset_index()
avg2 = avg2.rename(columns={'pts': 'avg_score'})

In [29]:
combined = pd.merge(combined, avg2,  how='left', left_on=['year','team_abbreviation'], right_on = ['year','team_abbreviation'])
combined['score_diff'] = combined['pts'] - combined['avg_score']

In [30]:
avg_diff = combined.groupby(['official_id', 'team_abbreviation'])['score_diff'].mean().reset_index()

In [31]:
num_games = combined.groupby(['official_id', 'team_abbreviation']).size().reset_index(name='num_games')

referee_names = combined[['official_id', 'first_name', 'last_name']].drop_duplicates()
referee_names['referee_name'] = referee_names['first_name'] + ' ' + referee_names['last_name']
referee_names = referee_names[['official_id', 'referee_name']].drop_duplicates()

avg_diff = pd.merge(avg_diff, num_games,  how='left', on=['official_id', 'team_abbreviation'])
avg_diff = pd.merge(avg_diff, referee_names, how='left', on='official_id')
avg_diff['abs_score_difference'] = avg_diff['score_diff'].abs()

In [32]:
combined[(combined['team_abbreviation'] == 'NJN') & (combined['official_id'] == '202049')]

,team_abbreviation,game_id,game_date,official_id,team_id,pts,first_name,last_name,year,avg_score,score_diff
29220,NJN,0021100360,2012-02-06,202049,1610612751,87,Ben,Taylor,2012,93.494318,-6.494318


In [33]:
avg_diff

,official_id,team_abbreviation,score_diff,num_games,referee_name,abs_score_difference
0,101283,ATL,0.164631,61,Brian Forte,0.164631
1,101283,BKN,-0.919404,40,Brian Forte,0.919404
2,101283,BOS,-0.895299,63,Brian Forte,0.895299
3,101283,CHA,-2.453807,55,Brian Forte,2.453807
4,101283,CHI,-1.350248,59,Brian Forte,1.350248
...,...,...,...,...,...,...
2847,2882,SAS,1.309160,61,Sean Wright,1.309160
2848,2882,SEA,-6.851104,10,Sean Wright,6.851104
2849,2882,TOR,1.346268,65,Sean Wright,1.346268
2850,2882,UTA,-0.349890,56,Sean Wright,0.349890


In [34]:
avg_diff_30 = avg_diff[avg_diff['num_games'] > 30]
avg_diff_30 = avg_diff_30.sort_values(by='abs_score_difference', ascending=False)

In [35]:
avg_diff_30

,official_id,team_abbreviation,score_diff,num_games,referee_name,abs_score_difference
2536,2148,NYK,7.457746,49,Mark Ayotte,7.457746
2483,204058,WAS,6.929190,33,Gediminas Petraitis,6.929190
2232,202041,POR,5.955241,39,Kevin Scott,5.955241
1206,1200,LAL,5.938039,35,Greg Willard,5.938039
206,1150,BOS,5.602471,32,Jimmy Clark,5.602471
...,...,...,...,...,...,...
1457,1662,CHA,0.005276,69,Bill Kennedy,0.005276
512,1162,CLE,0.003255,81,Scott Foster,0.003255
636,1167,PHX,-0.002485,36,Steve Javie,0.002485
671,1168,PHI,0.001987,51,David Jones,0.001987


In [36]:

diagram2 = avg_diff_30.head(10)[['team_abbreviation', 'referee_name', 'num_games', 'score_diff', 'abs_score_difference']]

diagram2 = diagram2.rename(columns={
    'team_abbreviation': 'Team',
    'num_games': 'Games',
    'score_diff': 'Difference',
    'abs_score_difference': 'Absolute Difference'
})
diagram2

,Team,referee_name,Games,Difference,Absolute Difference
2536,NYK,Mark Ayotte,49,7.457746,7.457746
2483,WAS,Gediminas Petraitis,33,6.929190,6.929190
2232,POR,Kevin Scott,39,5.955241,5.955241
1206,LAL,Greg Willard,35,5.938039,5.938039
206,BOS,Jimmy Clark,32,5.602471,5.602471
644,WAS,Steve Javie,31,-5.506602,5.506602
2222,MIL,Kevin Scott,43,5.454770,5.454770
617,DAL,Steve Javie,38,-5.255949,5.255949
2765,LAL,Olandis Poole,32,5.251950,5.251950
2293,ORL,Scott Twardoski,37,-5.110032,5.110032


In [37]:
diagram2.to_csv('diagram2.csv')

In [38]:
Ayotte_Games = combined[(combined['first_name']=='Mark') & (combined['last_name']=='Ayotte') & ((combined['team_abbreviation']=='NYK'))][['game_date','pts']]
NYK = combined[combined['team_abbreviation']=='NYK'][['game_date','pts']]

Ayotte_Games = Ayotte_Games.drop_duplicates()
NYK = NYK.drop_duplicates()

NYK['game_date'] = pd.to_datetime(NYK['game_date'])
Ayotte_Games['game_date'] = pd.to_datetime(Ayotte_Games['game_date'])

In [39]:
Ayotte_Games.to_csv('Ayotte_Games.csv')
NYK.to_csv('NYK.csv')

In [40]:
H = combined[(combined['last_name']=='Haskill')][['game_date','pts','official_id','first_name','last_name']]
H

,game_date,pts,official_id,first_name,last_name


In [41]:
# Load the data
data = pd.read_csv('combined_all3.csv')

In [42]:
# List of columns that contain statistics
stats_columns = ['fgm', 'fg_pct', 'fg3m', 'fg3_pct', 'ftm', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

# Group by referee and aggregate the original data
aggregated_raw_data = data.groupby(['first_name', 'last_name', 'official_id']).agg({**{col: 'mean' for col in stats_columns}, 'game_date': 'count'}).reset_index()
aggregated_raw_data = aggregated_raw_data.rename(columns={'game_date': 'num_games'})

In [43]:
aggregated_raw_data

,first_name,last_name,official_id,fgm,fg_pct,fg3m,fg3_pct,ftm,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,num_games
0,Aaron,Smith,1626302,40.705247,0.464247,11.689815,0.362363,17.341049,0.774860,10.375000,33.841049,44.216049,24.270062,7.743827,4.813272,14.092593,19.495370,110.441358,648
1,Andy,Nagy,1627534,41.215827,0.468018,12.449640,0.363888,17.118705,0.782453,10.237410,33.910072,44.147482,25.046763,7.654676,4.910072,13.935252,19.251799,112.000000,278
2,Anthony,Jordan,1169,35.318750,0.450831,5.409375,0.367269,18.962500,0.754603,11.500000,29.278125,40.778125,21.071875,7.809375,4.790625,15.059375,21.768750,95.009375,320
3,Ashley,Moyer-Gleich,1628953,40.943038,0.467633,11.943038,0.364453,17.655063,0.775310,10.031646,34.000000,44.031646,24.721519,7.765823,4.901899,14.417722,19.825949,111.484177,316
4,Ben,Taylor,202049,39.875746,0.464142,10.479125,0.357178,18.105368,0.768274,10.392644,33.082505,43.475149,23.740557,7.552684,4.727634,14.287276,20.724652,108.335984,1006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Tyler,Mirkovich,1628482,41.062500,0.469375,12.500000,0.348187,20.500000,0.793937,11.187500,32.750000,43.937500,24.562500,7.562500,4.312500,14.562500,21.812500,115.125000,16
152,Tyler,Ricks,1627539,40.200000,0.458333,11.800000,0.352267,15.266667,0.783867,10.266667,34.333333,44.600000,23.833333,8.266667,4.933333,13.466667,17.900000,107.466667,30
153,Violet,Palmer,1185,36.562298,0.452925,6.356796,0.355727,19.029935,0.759632,11.254045,30.619741,41.873786,21.412621,7.499191,4.999191,14.676375,21.545307,98.511327,1236
154,Vladimir,Voyard-Tadal,1626303,38.000000,0.448250,8.750000,0.335000,18.250000,0.764750,9.750000,32.750000,42.500000,22.000000,7.375000,4.250000,13.750000,21.000000,103.000000,8


In [44]:
# Normalize the aggregated data
aggregated_raw_data[stats_columns] = (aggregated_raw_data[stats_columns] - aggregated_raw_data[stats_columns].mean()) / aggregated_raw_data[stats_columns].std()

# Create a dataframe for means and standard deviations
means = data[stats_columns].mean()
stds = data[stats_columns].std()
mean_std_df = pd.DataFrame({
    'first_name': ['MEAN', 'STD_DEV'],
    'last_name': ['', ''],
    'official_id': ['', ''],
    **{col: [means[col], stds[col]] for col in stats_columns},
    'num_games': ['', '']
})

# Concatenate the mean_std_df with the aggregated raw data to create the final table
final_table_corrected = pd.concat([mean_std_df, aggregated_raw_data], ignore_index=True)
# Split the dataframe
top_rows = final_table_corrected.iloc[:2]
remaining_rows = final_table_corrected.iloc[2:]

# Sort the remaining rows
sorted_remaining_rows = remaining_rows.sort_values(by='num_games', ascending=False).reset_index(drop=True)

# Concatenate them back together
final_table_sorted = pd.concat([top_rows, sorted_remaining_rows], ignore_index=True)

In [45]:
stats_columns

['fgm',
 'fg_pct',
 'fg3m',
 'fg3_pct',
 'ftm',
 'ft_pct',
 'oreb',
 'dreb',
 'reb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts']

In [46]:
data[(data['last_name']=='Bernhardt')]

,game_date,team_abbrevation,official_id,first_name,last_name,fgm,fg_pct,fg3m,fg3_pct,ftm,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts
9,1996-11-19 00:00:00,ORL,1144,Ted,Bernhardt,27,0.355,6,0.333,28,0.757,7,24,31,14,10,3,17,23,88
37,1997-02-21 00:00:00,LAL,1144,Ted,Bernhardt,37,0.514,9,0.450,16,0.667,10,30,40,29,5,14,15,19,99
193,1999-12-15 00:00:00,VAN,1144,Ted,Bernhardt,37,0.430,6,0.400,26,0.743,12,43,55,22,6,6,17,30,106
223,2000-04-02 00:00:00,LAL,1144,Ted,Bernhardt,39,0.513,4,0.364,24,0.828,10,40,50,31,4,9,8,13,106
244,2000-11-11 00:00:00,IND,1144,Ted,Bernhardt,35,0.443,6,0.375,18,0.643,14,41,55,24,8,9,17,23,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74522,2005-11-02 00:00:00,GSW,1144,Ted,Bernhardt,39,0.419,3,0.300,16,0.762,20,27,47,21,12,3,22,35,97
74552,2005-11-04 00:00:00,DEN,1144,Ted,Bernhardt,29,0.333,3,0.231,7,0.350,16,28,44,20,7,12,19,18,68
74609,2005-11-06 00:00:00,PHX,1144,Ted,Bernhardt,44,0.444,7,0.438,23,0.793,20,26,46,28,9,2,10,18,118
74652,2005-11-09 00:00:00,IND,1144,Ted,Bernhardt,34,0.459,3,0.273,19,0.679,14,33,47,19,7,8,22,30,90


In [47]:
PCA = remaining_rows[remaining_rows['num_games'] > 100]

In [54]:
PCA.to_csv('PCA.csv')

In [49]:
pca_results = pd.read_csv('pca_results.csv')

In [50]:
pca_results['full_name'] = pca_results['first_name'] + " " + pca_results['last_name']
pca_results = pca_results.drop(columns=['X'])

In [51]:
pca_results

,first_name,last_name,official_id,fgm,fg_pct,fg3m,fg3_pct,ftm,ft_pct,oreb,...,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,full_name
0,Aaron,Smith,1626302,1.103713,0.554498,1.157707,0.438085,-0.604474,0.538783,-0.677941,...,-0.020794,0.364523,-0.148711,-0.171852,0.121561,0.040170,0.020091,1.083321e-15,-1.361417e-15,Aaron Smith
1,Andy,Nagy,1627534,1.330008,0.824532,1.425564,0.492457,-0.751356,0.846816,-0.821212,...,0.586821,0.514440,-0.159534,0.066602,0.017301,-0.046995,-0.089570,-1.423162e-15,-5.438130e-16,Andy Nagy
2,Anthony,Jordan,1169,-1.283641,-0.406151,-1.056304,0.612911,0.466667,-0.282908,0.493510,...,-0.028578,0.462150,0.373530,0.283533,-0.245785,0.109823,-0.024403,9.706421e-16,1.655327e-15,Anthony Jordan
3,Ashley,Moyer-Gleich,1628953,1.209105,0.796958,1.246974,0.512557,-0.397034,0.557060,-1.035472,...,-0.033151,-0.136467,0.105540,-0.142203,-0.057785,-0.119803,-0.024542,-8.989549e-17,4.269050e-16,Ashley Moyer-Gleich
4,Ben,Taylor,202049,0.736069,0.546996,0.730909,0.253331,-0.099560,0.271658,-0.659568,...,-0.231120,-0.178434,-0.148344,0.088826,0.050850,-0.037981,-0.005484,-5.281223e-17,-1.383846e-15,Ben Taylor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Tony,Brown,2529,-0.260638,0.061412,-0.244741,0.357308,-0.090042,0.009348,-0.225627,...,-0.404798,0.157891,0.273631,0.176630,0.034481,-0.057717,-0.000708,-2.208621e-15,-1.060248e-15,Tony Brown
113,Tre,Maddox,202027,0.574510,0.170314,0.464893,0.059465,-0.498329,0.115015,-0.340960,...,-0.018188,-0.282127,-0.201343,0.045838,0.090687,-0.062778,0.049826,-2.738838e-15,-1.698637e-16,Tre Maddox
114,Tyler,Ford,204059,0.835461,0.555943,0.982358,0.338512,0.040722,0.513569,-0.889164,...,-0.109282,-0.026451,0.350648,-0.022471,-0.060286,-0.012889,-0.012517,1.344159e-15,-1.101542e-15,Tyler Ford
115,Violet,Palmer,1185,-0.732487,-0.256242,-0.722315,0.201641,0.511215,-0.078919,0.237400,...,0.269791,0.067530,0.426316,-0.123065,-0.024044,-0.057706,0.011672,-2.985460e-16,-6.954927e-16,Violet Palmer


In [52]:
# Make Bokeh display charts in the notebook
output_notebook()

# Convert the dataframe to a ColumnDataSource for Bokeh
source = ColumnDataSource(pca_results)

# Create a new figure
p = figure(width=800, height=600,
           title="PCA Results: PC1 vs PC2",
           x_axis_label="PC1",
           y_axis_label="PC2")

# Add a scatter plot to the figure
p.circle(x="PC1", y="PC2", size=8, color="blue", alpha=0.6, source=source)

# Customize hover tool
hover = HoverTool()
hover.tooltips = [("Referee", "@full_name"),
                  ("PC1", "@PC1"),
                  ("PC2", "@PC2")]
p.add_tools(hover)

# Draw lines on the plot at x=0 and y=0
min_value_x, max_value_x = min(pca_results["PC1"]), max(pca_results["PC1"])
min_value_y, max_value_y = min(pca_results["PC2"]), max(pca_results["PC2"])

p.line([0, 0], [min_value_y, max_value_y], line_width=2, color="black")  # Line for y=0
p.line([min_value_x, max_value_x], [0, 0], line_width=2, color="black")  # Line for x=0

# Adjust fonts and styles for a more polished look
p.title.text_font_size = '16pt'
p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

# Adjust the figure for darker axes lines
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Make axes lines darker
p.xaxis.axis_line_width = 2
p.yaxis.axis_line_width = 2

# Display the figure
show(p)

Loading BokehJS ...

In [53]:
s = 'abc 123'
t = list(s)
t

['a', 'b', 'c', ' ', '1', '2', '3']